In [33]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
import os
from sklearn.metrics import mean_squared_error
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means
from skimage.measure import compare_ssim as ssim
from random import shuffle
import math
from keras.models import model_from_json
from keras.layers import *
from keras.models import Model
from keras import regularizers
from keras import backend as K
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint
from keras.layers.core import Lambda
import pickle

from utilities import *
%matplotlib inline


In [3]:
def readcifar(path) : 
    with open(path, 'rb') as f:
        train_set= pickle.load(f)
    return train_set["data"]

In [5]:
raw_train_set1 = readcifar('data/cifar-10-batches-py/data_batch_1')
raw_train_set2 = readcifar('data/cifar-10-batches-py/data_batch_2')
raw_train_set3 = readcifar('data/cifar-10-batches-py/data_batch_3')
raw_train_set4 = readcifar('data/cifar-10-batches-py/data_batch_4')
raw_train_set5 = readcifar('data/cifar-10-batches-py/data_batch_5')

In [6]:
raw_train_set = np.concatenate((raw_train_set1,raw_train_set2,raw_train_set3,raw_train_set4,raw_train_set5))

In [7]:
raw_train_set.shape

(50000, 3072)

In [8]:
def preprocess(images) :
    images = images.astype('float32')/255.
    images = np.reshape(images, (images.shape[0],32,32,3))
    return images

In [9]:
train_set = preprocess(raw_train_set)

In [10]:
train_set.shape

(50000, 32, 32, 3)

In [11]:
def visualise_cifar10():
    from six.moves import cPickle 

    f = open('data/cifar-10-batches-py/data_batch_1', 'rb')
    datadict = cPickle.load(f)
    f.close()
    X = datadict["data"] 
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
    Y = np.array(Y)

    #Visualizing CIFAR 10
    fig, axes1 = plt.subplots(5,5,figsize=(3,3))
    for j in range(5):
        for k in range(5):
            i = np.random.choice(range(len(X)))
            axes1[j][k].set_axis_off()
            axes1[j][k].imshow(X[i:i+1][0])
            
#Display last 'n' images from object 'images'
def display_colour_images(images,n,sizeX,sizeY):
    size = images.shape[0]
    plt.figure(figsize=(20, 2))
    for i in range(1,n):
        ax = plt.subplot(1, n, i)
        plt.imshow(images[size - i].reshape(sizeX, sizeY,3))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [149]:
#visualise_cifar10()

RuntimeError: Could not create write struct

In [10]:
display_colour_images(train_set,10,32,32)

RuntimeError: Could not create write struct

In [12]:
#Adding noise to images
def add_noise(images):
    #x_train = np.reshape(x_train, (len(x_train), 64*64))  # adapt this if using `channels_first` image data format
    #x_test = np.reshape(x_test, (len(x_test), 64*64))  # adapt this if using `channels_first` image data format
    batch = images.shape[0]//4;
    noise1 = gaussian_noise(images[0:batch],0,1,0.1)
    noise2 = gaussian_noise(images[batch:2*batch],0,1,0.5)
    noise3 = gaussian_noise(images[2*batch:3*batch],0,2,0.2)
    noise4 = gaussian_noise(images[3*batch:],0,3,0.1)
    
    noisy_set = []
    for data in [noise1,noise2,noise3,noise4]:
        for image in data:
            noisy_set.append(image)
    
    return np.array(noisy_set)
   
#Shuffle the noisy image ground truth pair to randomize the noise distribution in the dataset
def pair_shuffle(images,noisy_set):
    image_pair = []
    for i in range(images.shape[0]):
        image_pair.append((images[i],noisy_set[i]))
    shuffle(image_pair)
    
    ground_truth=[]
    noisy_images = []
    for i in range(images.shape[0]):
        ground_truth.append(image_pair[i][0])
        noisy_images.append(image_pair[i][1])
    return np.array(ground_truth), np.array(noisy_images)

In [13]:
#Shuffling and adding noise to the dataset
shuffle(train_set)
#Getting the noisy image set
noisy_set = add_noise(train_set)
ground_truth,noisy_images = pair_shuffle(train_set,noisy_set)

In [14]:
#Split into training and cross validation
train_size = int(ground_truth.shape[0]*0.8)
x_train = ground_truth[0:train_size]
x_train_noisy = noisy_images[0:train_size]
x_test = ground_truth[train_size:]
x_test_noisy = noisy_images[train_size:]
print (x_train_noisy.shape)
print (x_test_noisy.shape)

(40000, 32, 32, 3)
(10000, 32, 32, 3)


In [15]:
#Ground truth
"""display_images(ground_truth,10)"""

'display_images(ground_truth,10)'

In [16]:
#Noisy images
"""display_images(noisy_images,10)"""

'display_images(noisy_images,10)'

In [34]:
lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
csv_logger = CSVLogger('./models/autoencoder.csv')
early_stopper = EarlyStopping(min_delta=0.001,patience=30)
model_checkpoint = ModelCheckpoint('./models/autoencoder.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [35]:
#Defining the model

def get_simple_autoencoder_model(model_path=None):
    
    if(model_path is None):
        autoencoder = None
    else:
        autoencoder = read_model_json(model_path) 
    
    if(autoencoder is None):
        input_img = Input(shape=x_train_noisy[0].shape)  # adapt this if using `channels_first` image data format

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

        autoencoder = Model(input_img, decoded)
        autoencoder.compile(optimizer='sgd', loss='mean_squared_error',metrics = ['accuracy'])

    print (autoencoder.summary())
    return autoencoder

In [36]:
#Training the model
autoencoder = get_simple_autoencoder_model()
autoencoder.fit(x_train_noisy, x_train,
                epochs=50,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0, write_graph=True), model_checkpoint])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 8, 8, 64)          36928     
_________________________________________________________________
up_sampling2d_17 (UpSampling (None, 16, 16, 64)        0         
__________

40000/40000 [==============================] - 99s - loss: 0.0250 - acc: 0.4043 - val_loss: 0.0251 - val_acc: 0.4055
Epoch 23/50
40000/40000 [==============================] - 99s - loss: 0.0249 - acc: 0.4050 - val_loss: 0.0249 - val_acc: 0.4062
Epoch 24/50
40000/40000 [==============================] - 99s - loss: 0.0248 - acc: 0.4058 - val_loss: 0.0249 - val_acc: 0.4067
Epoch 25/50
40000/40000 [==============================] - 99s - loss: 0.0247 - acc: 0.4065 - val_loss: 0.0247 - val_acc: 0.4079
Epoch 26/50
40000/40000 [==============================] - 99s - loss: 0.0246 - acc: 0.4072 - val_loss: 0.0247 - val_acc: 0.4081
Epoch 27/50
40000/40000 [==============================] - 99s - loss: 0.0245 - acc: 0.4078 - val_loss: 0.0245 - val_acc: 0.4092
Epoch 28/50
40000/40000 [==============================] - 99s - loss: 0.0244 - acc: 0.4084 - val_loss: 0.0245 - val_acc: 0.4100
Epoch 29/50
40000/40000 [==============================] - 99s - loss: 0.0243 - acc: 0.4090 - val_loss: 0.024

40000/40000 [==============================] - 99s - loss: 0.0232 - acc: 0.4162 - val_loss: 0.0232 - val_acc: 0.4171
Epoch 49/50
40000/40000 [==============================] - 99s - loss: 0.0231 - acc: 0.4164 - val_loss: 0.0232 - val_acc: 0.4178
Epoch 50/50
40000/40000 [==============================] - 99s - loss: 0.0231 - acc: 0.4168 - val_loss: 0.0231 - val_acc: 0.4183


In [42]:
#Function to get saved keras model
def read_model_json(jsonfilePath,h5filePath):
    try:
        json_file = open(jsonfilePath, 'r')
        print json_file
        loaded_model_json = json_file.read()
        json_file.close()
        print "hello"
        loaded_model = model_from_json(loaded_model_json)
         
        # load weights into new model
        loaded_model.load_weights(h5filePath)

        return loaded_model
    except:
        return None

In [43]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    gateFactor = Input(tensor = K.variable([gatePercentageFactor]))
    fractionG = Lambda(lambda x: x[0]*x[1])([inputLayer,gateFactor])
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return gateFactor,fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
        print model_path[0],model_path[1]
    if(sym_autoencoder is None):
        input_img = Input(shape=(32,32,3))  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, (4, 4), activation='relu', padding='same')(input_img)
        gf1,fg1,c1 = get_gated_connections(0.1,x1)

        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, (4, 4), activation='relu', padding='same')(x) 
        gf2,fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, (4, 4), activation='relu', padding='same')(x) 
        gf3,fg3,c3 = get_gated_connections(0.3,x3)

        x = MaxPooling2D((2, 2), padding='same')(x3)
        x4 = Conv2D(256, (4, 4), activation='relu', padding='same')(x) 
        gf4,fg4,c4 = get_gated_connections(0.4,x4)

        x = MaxPooling2D((2, 2), padding='same')(x4)
        x5 = Conv2D(512, (4, 4), activation='relu', padding='same')(x) 

        x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, (4, 4), activation='relu', padding='same')(x) 
        jt4 = Add()([y1,c4])
        x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, (4, 4), activation='relu', padding='same')(x) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(3, (4, 4), activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model([input_img,gf1,gf2,gf3,gf4],layers)
        sym_autoencoder.compile(optimizer='sgd', loss = 'mean_squared_error', metrics = ['accuracy'])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [44]:
sym_autoencoder = get_cnn_dsc_architecture()

Model created
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_18 (InputLayer)            (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_46 (Conv2D)               (None, 32, 32, 64)    3136        input_18[0][0]                   
____________________________________________________________________________________________________
input_19 (InputLayer)            (1,)                  0                                            
____________________________________________________________________________________________________
lambda_17 (Lambda)               (None, 32, 32, 64)    0           conv2d_46[0][0]                  
                                                                   input_19[0

In [45]:
model_checkpoint1 = ModelCheckpoint('./models/sym_autoencoder.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [46]:
sym_autoencoder.fit(x_train_noisy, x_train,
                epochs=50,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/sym_autoencoder', 
                                       histogram_freq=0,
                                       write_graph=True),model_checkpoint1])

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 470s - loss: 0.0266 - acc: 0.3827 - val_loss: 0.0192 - val_acc: 0.4124
Epoch 2/50
40000/40000 [==============================] - 469s - loss: 0.0183 - acc: 0.4291 - val_loss: 0.0177 - val_acc: 0.4422
Epoch 3/50
40000/40000 [==============================] - 469s - loss: 0.0175 - acc: 0.4485 - val_loss: 0.0173 - val_acc: 0.4541
Epoch 4/50
40000/40000 [==============================] - 469s - loss: 0.0172 - acc: 0.4574 - val_loss: 0.0171 - val_acc: 0.4592
Epoch 5/50
40000/40000 [==============================] - 469s - loss: 0.0170 - acc: 0.4629 - val_loss: 0.0169 - val_acc: 0.4643
Epoch 6/50
40000/40000 [==============================] - 469s - loss: 0.0169 - acc: 0.4666 - val_loss: 0.0168 - val_acc: 0.4683
Epoch 7/50
40000/40000 [==============================] - 468s - loss: 0.0168 - acc: 0.4695 - val_loss: 0.0167 - val_acc: 0.4723
Epoch 8/50
40000/40000 [=======================

40000/40000 [==============================] - 468s - loss: 0.0161 - acc: 0.4829 - val_loss: 0.0161 - val_acc: 0.4852
Epoch 28/50
40000/40000 [==============================] - 468s - loss: 0.0161 - acc: 0.4830 - val_loss: 0.0161 - val_acc: 0.4856
Epoch 29/50
40000/40000 [==============================] - 468s - loss: 0.0161 - acc: 0.4833 - val_loss: 0.0160 - val_acc: 0.4842
Epoch 30/50
40000/40000 [==============================] - 468s - loss: 0.0160 - acc: 0.4834 - val_loss: 0.0160 - val_acc: 0.4856
Epoch 31/50
40000/40000 [==============================] - 468s - loss: 0.0160 - acc: 0.4836 - val_loss: 0.0160 - val_acc: 0.4853
Epoch 32/50
40000/40000 [==============================] - 468s - loss: 0.0160 - acc: 0.4837 - val_loss: 0.0160 - val_acc: 0.4834
Epoch 33/50
40000/40000 [==============================] - 468s - loss: 0.0160 - acc: 0.4839 - val_loss: 0.0160 - val_acc: 0.4812
Epoch 34/50
40000/40000 [==============================] - 468s - loss: 0.0160 - acc: 0.4840 - val_los

In [47]:
x_train_noisy.shape

(40000, 32, 32, 3)

In [49]:
save_model_as_json(sym_autoencoder,'CNNDSC_CIFAR')

In [48]:
#Getting images to test
def get_testing_images(filePath):
    test_images = read_dataset(filePath)
    test_images = np.array(test_images)
    test_images = test_images.astype('float32')/255.
    test_images = np.reshape(test_images, (test_images.shape[0],32,32,3))
    noisy_test_images = add_noise(test_images)
    print (noisy_test_images.shape)
    return (test_images,noisy_test_images)

In [150]:
raw_test_images = readcifar('data/cifar-10-batches-py/test_batch')
test_images = preprocess(test_images)
print(test_images.shape)
noisy_test_images = add_noise(test_images)

(10000, 32, 32, 3)


In [51]:
#Denoising using CNN symmetric Autoencoders
out_sym_autoencoder = sym_autoencoder.predict(noisy_test_images,verbose=1)
print (out_sym_autoencoder.shape)

10000/10000 [==============================] - 34s    
(10000, 32, 32, 3)


In [52]:
#Denoising using Autoencoders
out_autoencoder = autoencoder.predict(noisy_test_images,verbose=1)
print (out_autoencoder.shape)

10000/10000 [==============================] - 7s     
(10000, 32, 32, 3)


In [57]:
"""
display_images(test_images,10)
display_images(noisy_test_images,10)
display_images(out,10)
"""

'\ndisplay_images(test_images,10)\ndisplay_images(noisy_test_images,10)\ndisplay_images(out,10)\n'

In [78]:
from sklearn.metrics import mean_squared_error
def get_psnr(imageA,imageB):
    maxI = 1.0
    try:
        return 20*math.log10(maxI) - 10*math.log10(mean_squared_error(imageA.flatten(),imageB.flatten()))
    except:
        return 20*math.log10(maxI)

def get_psnr_result(x_test, out):
    psnr_sum = 0
    for i in range(out.shape[0]):
        psnr_sum += get_psnr(x_test[i].reshape(32,32,3),out[i].reshape(32,32,3))
        
    return 1.0*psnr_sum/out.shape[0];

def get_ssim_result(originalSet,noisySet):
    ssim_sum = 0
    originalSet = originalSet.reshape(originalSet.shape[0],32,32,3)
    noisySet = noisySet.reshape(noisySet.shape[0],32,32,3)
    for i in range(originalSet.shape[0]):
        ssim_sum += ssim(originalSet[i], noisySet[i],data_range=originalSet[i].max() - noisySet[i].min(), multichannel=True)
    return 1.0*ssim_sum/originalSet.shape[0]

In [70]:
import pybm3d

In [161]:
def bm3d_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32,3)
    noisy_image = noisy_image*255.0
    denoised = []
    count = 1
    for i in range(noisy_image.shape[0]):
        den_img = pybm3d.bm3d.bm3d(noisy_image[i])
        denoised.append(den_img)
        if (count%10 == 0):
            print (str(count)+ "images denoised")
        count+=1
        
    return np.array(denoised)

def nlm_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32,3)
    noisy_image = noisy_image*255.0
    denoised = []
    count = 1
    
    for image in noisy_image:
        denoised_image = denoise_nl_means(image, 7, 11, 0.5,multichannel = True)
        denoised.append(denoised_image)
        if(count%100 == 0) :
            print(str(count)+" images denoised")
        count+=1
    return np.array(denoised)

In [79]:
print (test_images.shape,out_autoencoder.shape)
print (mean_squared_error(test_images[0].flatten(),out_autoencoder[0].flatten()))
get_psnr_result(out_autoencoder,test_images)

((10000, 32, 32, 3), (10000, 32, 32, 3))
0.017626


16.849217702825033

In [80]:
print (test_images.shape,out_sym_autoencoder.shape)
print (mean_squared_error(test_images[0].flatten(),out_sym_autoencoder[0].flatten()))
get_psnr_result(out_sym_autoencoder,test_images)

((10000, 32, 32, 3), (10000, 32, 32, 3))
0.0088522


18.591258702624295

In [61]:
import bm3d

In [162]:
noisy_test_images.shape
bm3d_out = bm3d_denoise(noisy_test_images)

TypeError: run_bm3d_wrap() takes at least 2 positional arguments (1 given)

In [ ]:
bm3d_out_norm = bm3d_out.astype('float32')/255.0

In [ ]:
get_psnr_result(bm3d_out_norm,test_images)

In [67]:
print(noisy_test_images.shape)
nlm_out = nlm_denoise(noisy_test_images)
nlm_out = nlm_out.astype('float32')/255.0

(10000, 32, 32, 3)
100 images denoised
200 images denoised
300 images denoised
400 images denoised
500 images denoised
600 images denoised
700 images denoised
800 images denoised
900 images denoised
1000 images denoised
1100 images denoised
1200 images denoised
1300 images denoised
1400 images denoised
1500 images denoised
1600 images denoised
1700 images denoised
1800 images denoised
1900 images denoised
2000 images denoised
2100 images denoised
2200 images denoised
2300 images denoised
2400 images denoised
2500 images denoised
2600 images denoised
2700 images denoised
2800 images denoised
2900 images denoised
3000 images denoised
3100 images denoised
3200 images denoised
3300 images denoised
3400 images denoised
3500 images denoised
3600 images denoised
3700 images denoised
3800 images denoised
3900 images denoised
4000 images denoised
4100 images denoised
4200 images denoised
4300 images denoised
4400 images denoised
4500 images denoised
4600 images denoised
4700 images denoised
480

In [68]:
get_psnr_result(nlm_out,test_images)

12.904738896870535

In [65]:
get_ssim_result(test_images,out_autoencoder)

0.5642729314306093

In [66]:
get_ssim_result(test_images,out_sym_autoencoder)

0.718734871276698

In [ ]:
get_ssim_result(test_images,bm3d_out_norm)

In [69]:
get_ssim_result(test_images,nlm_out)

0.49438484841716263

In [101]:
import scipy.misc


In [157]:
raw_test_images = readcifar('data/cifar-10-batches-py/test_batch')
raw_test_images = np.array(raw_test_images)
raw_test_images_re = raw_test_images.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
scipy.misc.toimage(raw_test_images_re[1], cmin=0.0, cmax=255).save('groundTruth.jpg')

In [126]:
noisy_test_images = add_noise(raw_test_images_re)
noisy_test_images_re = noisy_test_images_re.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
scipy.misc.imsave('out_noisy.jpg', noisy_test_images_re[0])

In [99]:
cv2.imwrite('out_raw_image.png',raw_test_images_re[0])

True

In [156]:
test = raw_test_images_re[0] + np.random.normal(0,10,raw_test_images_re[0].shape)
print test.shape
scipy.misc.toimage(test, cmin=0.0, cmax=255).save('noisyImage.jpg')

(32, 32, 3)


In [92]:
test_images = read_dataset(filePath)
test_images = np.array(test_images)

NameError: name 'filePath' is not defined

In [145]:
np.random.normal(0,10,raw_test_images_re[0].shape).shape

(32, 32, 3)

In [153]:
scipy.misc.toimage(noisy_test_images[0]*255, cmin=0.0, cmax=255).save('noisyImage.jpg')

In [152]:
noisy_test_images[0]

array([[[ 0.24449301,  0.05683702,  0.03648469],
        [ 0.0772326 ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.0930394 ,  0.02182998],
        [ 0.        ,  0.09627139,  0.14737143],
        [ 0.1256676 ,  0.        ,  0.0246383 ]],

       [[ 0.07830135,  0.09261384,  0.02461535],
        [ 0.08843171,  0.        ,  0.        ],
        [ 0.02886679,  0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.10082165,  0.        ],
        [ 0.09477774,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.20705065]],

       [[ 0.        ,  0.        ,  0.03018796],
        [ 0.        ,  0.        ,  0.10510143],
        [ 0.        ,  0.        ,  0.        ],
        ..., 
        [ 0.00492154,  0.05645058,  0.        ],
        [ 0.11785186,  0.21180976,  0.01847803],
        [ 0.        ,  0.09241974,  0.        ]],

       ..., 
       [[ 0.        ,  0.        ,  0.12735681],
        

In [158]:
from keras.datasets import 

In [160]:
cifar10 = cifar10.load_data()

 29024256/170498071 [====>.........................] - ETA: 486s

KeyboardInterrupt: 